In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig, pipeline
import torch
import pandas as pd
device = "cuda:0"

In [2]:
load_model_name = "../result/DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13/best" # 학습된 모델 불러오기 가능
load_tokenizer_name = "../result/DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13"

models = AutoModelForCausalLM.from_pretrained(load_model_name, load_in_8bit=True)
tokenizers = AutoTokenizer.from_pretrained(load_tokenizer_name)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test_split.csv")

In [3]:
print(tokenizers.pad_token)
print(tokenizers.eos_token)

</s>
<|im_end|>


In [5]:
test.head(5)

,id,질문
0,TEST_000,방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?
1,TEST_000,원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
2,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
3,TEST_002,큐블럭의 단점을 알려주세요.
4,TEST_002,압출법 단열판을 사용하는 것의 장점은 무엇인가요?


In [37]:
import re
input_text = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?'

with open("../template/datavortex.txt", "r", encoding="utf-8") as file:
    content = file.read()

answer_start_index = content.find("<answer>")
content = content[:answer_start_index]
content = content.replace("<question>", input_text)

input_ids = tokenizers.encode(
            content, padding=False, max_length=256, return_tensors="pt", add_special_tokens=False
        )
print(content, "\n--------------\n")
print(content.replace("\n", "\\n"), "\n--------------\n")
print(input_ids)

### System:
당신은 사람들이 정보를 찾을 수 있도록 도와주는 인공지능 비서입니다.

### User:
방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?

### Assistant:
 
--------------

### System:\n당신은 사람들이 정보를 찾을 수 있도록 도와주는 인공지능 비서입니다.\n\n### User:\n방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?\n\n### Assistant:\n 
--------------

tensor([[  774,  2135, 28747,    13, 30287, 36771,   259, 34875, 32100,   259,
         42857,   259, 31161, 29189, 28705, 29151, 28705, 29604, 32342,   259,
         34243, 32792,   259, 29324, 30010, 35894,   259, 42177, 32184,   842,
            13,    13, 27332,  1247, 28747,    13, 30240, 30594, 28705, 43950,
         33722,   259, 43850, 32096,   259, 29433, 46747,   259, 30439, 32100,
           259, 32707, 28705, 29161, 28705, 30976, 29511, 28705, 30106, 30126,
         35338,  1550,    13,    13, 27332, 21631, 28747,    13]])


In [36]:
output_sequences = models.generate(
        input_ids=input_ids.to("cuda"),
        max_length=512,
        temperature=0.2,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

print(tokenizers.decode(output_sequences[0]))

### System:
당신은  사람들이  정보를  찾을 수 있도록  도와주는  인공지능  비서입니다 .
### User:
방청 페인트의  종류에는  어떤  것들이  있는 지 알고 계신가요 ?
### Assistant:
방청 페인트는  다양한 종류 로 분류됩니다 . 그중에서 도 주요 종류로는  광 명단 방청페인 트, 방색제 혼 합방청페인 트, 그리고  니켈  안료 방청페인트가  있습니다 . 이러한 방법들은  각자의  장점과 용도에  맞 게 선택되어  사용됩니다 .<|im_end|>


---

# start token이 잘못되어 보정

In [29]:
import json
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

In [23]:
with open(f'../config/datavortex_0304.json', 'r', encoding='utf-8') as file:
    CFG = json.load(file)

In [31]:
def submission(CFG, preds):
    model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
    nl = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['TEST_DATA']}")
    submit = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['SUBMISSION_DATA']}")
    submission_name = "DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13"
    nl["답변"] = preds
    nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_{submission_name}.csv', index=False)
    if len(nl) != len(submit):
        nl = (
            nl.groupby("id")["답변"]
            .apply(lambda x: " ".join(x.astype(str)))
            .reset_index()
        )
        preds = nl["답변"]
        nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_merge_{submission_name}.csv', index=False)
    pred_embeddings = model.encode(preds)
    print("Shape of Prediction Embeddings: ", pred_embeddings.shape)
    submit.iloc[:, 1:] = pred_embeddings
    submit.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/{submission_name}.csv', index=False)

In [33]:
infer_csv = pd.read_csv("../result/NL_DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13.csv")

In [34]:
preds = []
start_token = "### Assistant: "
for full_text in infer_csv['답변']:
    answer_start = full_text.find(start_token) + len(start_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace("\n", "")
    answer_only = re.sub(r'\s+', ' ', answer_only)
    answer_only = answer_only.replace("<|im_end|>", "")
    answer_only = answer_only.replace("</s>", "")
    preds.append(answer_only)

In [32]:
submission(CFG, preds)

Shape of Prediction Embeddings:  (130, 512)


/tmp/ipykernel_14650/1588965106.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.00908877  0.01005     0.04502014 -0.00518508 -0.00833127 -0.00477516
 -0.01209085  0.02485655 -0.02140034  0.05036298 -0.00598953  0.00886326
 -0.02716835  0.01467484 -0.02769513  0.0083735   0.00895131 -0.01003359
  0.02156547  0.0193863  -0.01874679  0.00368112 -0.01108478  0.00088468
  0.00010685 -0.0118179   0.01151926 -0.00662439  0.07739285  0.01117572
 -0.04620844 -0.01453778  0.01523224 -0.01420029  0.01825143  0.01580314
 -0.01436109 -0.00101008 -0.02678643  0.00189441  0.02905442 -0.01633028
  0.02632357 -0.01538142 -0.04429469 -0.01680868 -0.009291   -0.03870575
 -0.05463232  0.00674701 -0.04070652 -0.00568772 -0.00964655  0.00777236
  0.04294617  0.01937784  0.02833878  0.03319206 -0.06643791  0.03668335
 -0.00538973  0.00254746 -0.0106891   0.04580025 -0.05195896 -0.00046686
  0.00649229  0.00831351 -0.02179497 -0.